In [1]:
machine = 'mac'
machine = 'magny'

In [2]:
if machine == 'magny':
    basedir = "/hits/universe/GigaGalaxy/level4_MHD/"
    outdir = "/home/extmilan/masterthesis/files/"
elif machine == 'mac': 
    basedir = "/Users/smilanov/Desktop/Auriga/level4/"
    outdir = "/Users/smilanov/Documents/masterthesis/auriga_files/files/"
    
file = outdir + 'potential_params.txt'

In [3]:
from galpy.potential import NFWPotential as NFWpot
from galpy.potential import HernquistPotential as HEpot
from galpy.potential import DoubleExponentialDiskPotential as DEpot
from galpy.potential import MiyamotoNagaiPotential as MNpot
from galpy.potential import MN3ExponentialDiskPotential as MN3pot
from galpy.potential import MWPotential2014
from galpy.potential import evaluatePotentials, MiyamotoNagaiPotential, NFWPotential, HernquistPotential


from galpy.potential import evaluateDensities, evaluatePotentials
from galpy.potential import plotDensities
from galpy.potential import plotRotcurve
from galpy.util import bovy_conversion

import emcee

import numpy as np
from scipy import optimize as opt

from astropy import units

import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm

from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import copy

from areposnap.gadget import gadget_readsnap
from areposnap.gadget_subfind import load_subfind

from auriga_basics import *
from auriga_functions import *

import datetime
import random

%matplotlib inline

With eat_snap_and_fof we can read in simulations from snapnr 3.

Number of DM particles (does not change during simulation run): 34,898,087

Let's take a random sample which makes $0.5\%$:

$0.005 \cdot 34898087 \approx 175000$



Potential fitting function input values
---

- maximum R [Mpc] - def 2 * s.galrad
- mask to fit potential (default - dm)
    - 0: all
    - 1: gas
    - 2: DM
    - 3: stars
    
- number of particles
- fitting function (default - emcee1)
    - if emcee1 number of walkers, number of steps
- savefile
- R0 [kpc] - def 8kpc

parameters:
- plot
- save
- verbose


In [4]:
#_____function that sets-up galpy potential_____
def setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB, _REFR0_kpc):
    
    #test input:
    if (a_MND_kpc <= 0.) or (b_MND_kpc <= 0.) or (a_NFWH_kpc <= 0.) or (a_HB_kpc <= 0.) \
       or (n_MND <= 0.) or (n_NFWH <= 0.) or (n_HB <= 0.) or (n_MND >= 1.) or (n_NFWH >= 1.) or (n_HB >= 1.):
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The input parameters for the scaling profiles do not correspond to a physical potential.')
    if np.fabs(n_MND + n_NFWH + n_HB - 1.) > 1e-7:
        raise ValueError('Error in setup_galpy_potential: '+\
                         'The sum of the normalization does not add up to 1.')
        
    #trafo to galpy units:
    a_MND  = a_MND_kpc / _REFR0_kpc
    b_MND  = b_MND_kpc / _REFR0_kpc
    a_NFWH = a_NFWH_kpc / _REFR0_kpc
    a_HB   = a_HB_kpc / _REFR0_kpc
    
    #setup potential:
    disk = MiyamotoNagaiPotential(
                a = a_MND,
                b = b_MND,
                normalize = n_MND)
    halo = NFWPotential(
                a = a_NFWH,
                normalize = n_NFWH)
    bulge = HernquistPotential(
                a = a_HB,
                normalize = n_HB)
    return [disk, halo, bulge]


In [5]:
#_____function for MCMC, sum of error squares_____
def lnprob_MCMC(x, *args):
    
    # read fitting parameters:
    v0_kms     = x[0]
    a_MND_kpc  = x[1]
    b_MND_kpc  = x[2]
    a_NFWH_kpc = x[3]
    a_HB_kpc   = x[4]
    n_MND      = x[5]
    n_NFWH     = x[6]
    n_HB       = 1. - n_NFWH - n_MND
    
    # check if imput parameters are physical:
    if v0_kms <= 0: return -np.inf
    # (Note: all other parameters are checked during setting up the galpy potential.)
        
    #read data:
    R_kpc_data    = args[0]
    z_kpc_data    = args[1]
    pot_kms2_data = args[2]
    _REFR0_kpc    = args[3]
    
    # setup potential (and check if parameters are physical):
    try:
        pot_galpy_model = setup_galpy_potential(a_MND_kpc, b_MND_kpc, a_NFWH_kpc, a_HB_kpc, n_MND, n_NFWH, n_HB, _REFR0_kpc)
    except Exception as e:
        #uncomment for debugging:
        #print e.message
        #print x
        return -np.inf

    #calculate potential values at (R,z) for this potential:
    pot_kms2_model = evaluatePotentials(pot_galpy_model,
                                   R_kpc_data / _REFR0_kpc,
                                   z_kpc_data / _REFR0_kpc) * (v0_kms)**2
    
    #calculate sum of relative error squares:
    err = np.sum(-0.5 * (pot_kms2_data - pot_kms2_model)**2)
    return err

In [ ]:
def potential_fit(maxr_Mpc, _REFR0_kpc = 8., mask_num = 2, part_num = 1501, \
                  v0_kms_init = 150., a_MND_kpc_init = 6., b_MND_kpc_init = 1., \
                  a_NFWH_kpc_init = 10., a_HB_kpc_init = 2., n_MND_init = 0.2, n_NFWH_init = 0.6, \
                  fitroutine = 'emcee1', nwalkers = 50, nstep = 1000, \
                  file = file, res_arr = [], plot = False, save = False, verbose = False):
    
    ### selection masks
    # select ALL particles belonging to the main galaxy within maxr
    if mask_num == 0:
        mask, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) )
        parts_to_fit = 'all'
    # select GAS particles belonging to the main galaxy within maxr
    if mask_num == 1:
        mask, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) & (s.type == 0) )
        parts_to_fit = 'gas'
    # select DM particles belonging to the main galaxy within maxr
    if mask_num == 2:
        mask, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) & ((s.type == 1) + (s.type == 2) + (s.type == 3) ))
        parts_to_fit = 'DM'
    # select STAR particles belonging to the main galaxy within maxr
    if mask_num == 3:
        mask, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) & (s.type == 4) )
        parts_to_fit = 'star'
    
    if verbose == True:
        print('The potential is fitted to ' + str(part_num) + ' random selected ' + parts_to_fit + ' particles.')
   
    # select data with mask
    r_kpc = 1000. * s.r()[mask]
    (R_kpc, phi_rad, z_kpc), (vR_kms, vphi_kms, vz_kms) = get_cylindrical_vectors(s, sf, mask) 
    pot_kms2 = s.pot[mask]
    
    # draw random sample out of data
    try:
        rand_samp = np.array(random.sample(list(enumerate(pot_kms2)), part_num))
        rand_ind = rand_samp[:,0].astype(int)
        pot_kms2_data = rand_samp[:, 1]
        r_kpc_data = r_kpc[rand_ind]
        R_kpc_data = R_kpc[rand_ind]
        z_kpc_data = z_kpc[rand_ind]
    except ValueError:        
        pot_kms2_data = pot_kms2
        r_kpc_data = r_kpc
        R_kpc_data = R_kpc
        z_kpc_data = z_kpc
    
    if fitroutine == 'emcee1': # maybe implement diff ev but not now
        # _____prepare MCMC_____
        ndim, nwalkers, nstep = 7, nwalkers, nstep

        # initial values for fit:
        # init values as input of function
        initial_guess = np.array([v0_kms_init, a_MND_kpc_init, b_MND_kpc_init, a_NFWH_kpc_init, \
                                  a_HB_kpc_init, n_MND_init, n_NFWH_init])
        initial_guess_width = np.fabs(0.2 * initial_guess)
        p0 = np.array([np.random.randn(ndim) * np.array(initial_guess_width) + initial_guess \
                                                      for i in range(nwalkers)])
        # fiitting boundaries:
        boundaries = np.array([[0.,np.inf],[0.,np.inf],[0.,np.inf],[0.,np.inf],[0.,np.inf],[0.,1.],[0.,1.]])
        for pp in range(ndim):
            p0[(p0[:,pp] < boundaries[pp,0]),pp] = initial_guess[pp] + np.random.uniform() * 0.1
            p0[(p0[:,pp] > boundaries[pp,1]),pp] = initial_guess[pp] - np.random.uniform() * 0.1

        # _____run MCMC_____
        sampler = emcee.EnsembleSampler(nwalkers, ndim,
                                       lnprob_MCMC,
                                       args = [R_kpc_data, z_kpc_data, pot_kms2_data, _REFR0_kpc])
        sampler.run_mcmc(p0, nstep)

        # result:
        burnin = int(nwalkers / 2.) # check what this is used for. #1000
        final_samples = sampler.chain[:, burnin:, :].reshape((-1, ndim))
        chain_samples = sampler.chain

        median = np.median(final_samples,axis=0)
        std = np.std(final_samples,axis=0)

        v0_kms_bestfit     = median[0]
        a_MND_kpc_bestfit  = median[1]
        b_MND_kpc_bestfit  = median[2]
        a_NFWH_kpc_bestfit = median[3]
        a_HB_kpc_bestfit   = median[4]
        n_MND_bestfit      = median[5]
        n_NFWH_bestfit     = median[6]
        n_HB_bestfit       = 1. - (n_MND_bestfit + n_NFWH_bestfit)

        v0_kms_bestfit_err     = std[0]
        a_MND_kpc_bestfit_err  = std[1]
        b_MND_kpc_bestfit_err  = std[2]
        a_NFWH_kpc_bestfit_err = std[3]
        a_HB_kpc_bestfit_err   = std[4]
        n_MND_bestfit_err      = std[5]
        n_NFWH_bestfit_err     = std[6]
        n_HB_bestfit_err       = (n_MND_bestfit_err + n_NFWH_bestfit_err)

        
        
    # save potential parameters with errors and snapnr:
        pot_bestfit = np.array([v0_kms_bestfit, v0_kms_bestfit_err, a_MND_kpc_bestfit, a_MND_kpc_bestfit_err, \
                            b_MND_kpc_bestfit, b_MND_kpc_bestfit_err, a_NFWH_kpc_bestfit, a_NFWH_kpc_bestfit_err, \
                            a_HB_kpc_bestfit, a_HB_kpc_bestfit_err, n_MND_bestfit, n_MND_bestfit_err, \
                            n_NFWH_bestfit, n_NFWH_bestfit_err, n_HB_bestfit, n_HB_bestfit_err])
        
    res_arr = np.append(res_arr, pot_bestfit)
    #np.savetxt(file, pot_bestfit, fmt = '%10.15f')
    
    # generate potential measurements:
    pot_galpy_bestfit = setup_galpy_potential(a_MND_kpc_bestfit,b_MND_kpc_bestfit,
                                           a_NFWH_kpc_bestfit,a_HB_kpc_bestfit,
                                           n_MND_bestfit,n_NFWH_bestfit,n_HB_bestfit, _REFR0_kpc)
    pot_kms2_bestfit = evaluatePotentials(pot_galpy_bestfit,
                                       R_kpc_data/_REFR0_kpc,
                                       z_kpc_data/_REFR0_kpc) * (v0_kms_bestfit)**2
    if fitroutine == 'emcee1':
        sampler.reset()

    return res_arr

In [ ]:
#### path = /hits/universe/GigaGalaxy/level4_MHD/halo_24/output/*
level = 4

j = 0
for halo_number in [24]:  # range(1, 31):
    halodir = basedir+"halo_{0}/".format(halo_number)
    snappath = halodir+"output/"
    for snapnr in range(121,128,1):
        #print("level   : {0}".format(level))
        #print("halo    : {0}".format(halo_number))
        print("snapnr  : {0}".format(snapnr))
        print(datetime.datetime.now().time())
        #print("basedir : {0}".format(basedir))
        #print("halodir : {0}".format(halodir))
        #print("snappath: {0}\n".format(snappath))
        try:
            s, sf = eat_snap_and_fof(level, halo_number, snapnr, snappath, loadonlytype=[1,2,3], 
            haloid=0, galradfac=0.1, verbose=False, rotate_disk=False, do_rotation=False, use_principal_axis=False) 
        except KeyError:
            print('\n\n', snapnr, 'not read in.\n\n')
            continue

        # Clean negative and zero values of gmet to avoid RuntimeErrors
        # later on (e.g. dividing by zero)
        #s.data['gmet'] = np.maximum( s.data['gmet'], 1e-40 )
        #print('Maximum phase space extend of snapshot' + str(snapnr) + 'in Mpc:', np.max(s.r()))
        if j == 0:
            res_arr = potential_fit(maxr_Mpc = 2. * s.galrad, part_num = 175000)
            res_arr = np.insert(res_arr, 0, snapnr)
            j +=1
        else:
            res_arr = potential_fit(maxr_Mpc = 2. * s.galrad, part_num = 175000, res_arr=res_arr)
            res_arr = np.insert(res_arr, 17 * (snapnr - 121 ), snapnr)

        #print(res_arr)
        '''
        if (snapnr % 10) ==0:

            f = open(file, 'w') # existing file will be overwritten
            f.write("#snapnr\tv0_kms_bestfit\tv0_kms_bestfit_err\ta_MND_kpc_bestfit\ta_MND_kpc_bestfit_err\tb_MND_kpc_bestfit\tb_MND_kpc_bestfit_err\ta_NFWH_kpc_bestfit\ta_NFWH_kpc_bestfit_err\ta_HB_kpc_bestfit\ta_HB_kpc_bestfit_err\tn_MND_bestfit\tn_MND_bestfit_err\tn_NFWH_bestfit\tn_NFWH_bestfit_err\tn_HB_bestfit\tn_HB_bestfit_err\n")
            f.close()
            f = open(file,'ab') # open file in append and binary mode to make savetxt work and append
            np.savetxt(f,res_arr)
            f.close()
        '''

res_arr = res_arr.reshape(7,17) # instead of 4 number of simulations fitted

np.savetxt(outdir + 'safetyfile.txt', res_arr)

#f = open(file, 'w') # existing file will be overwritten
#f.write("#snapnr\tv0_kms_bestfit\tv0_kms_bestfit_err\ta_MND_kpc_bestfit\ta_MND_kpc_bestfit_err\tb_MND_kpc_bestfit\tb_MND_kpc_bestfit_err\ta_NFWH_kpc_bestfit\ta_NFWH_kpc_bestfit_err\ta_HB_kpc_bestfit\ta_HB_kpc_bestfit_err\tn_MND_bestfit\tn_MND_bestfit_err\tn_NFWH_bestfit\tn_NFWH_bestfit_err\tn_HB_bestfit\tn_HB_bestfit_err\n")
#f.close()
f = open(file,'ab') # open file in append and binary mode to make savetxt work and append
np.savetxt(f,res_arr)
f.close()


snapnr  : 121
11:39:48.692623
[ 32.04627228  30.8679924   34.56239319]
/home/extmilan/anaconda3/lib/python3.6/site-packages/emcee/ensemble.py:335: RuntimeWarning: invalid value encountered in subtract
  lnpdiff = (self.dim - 1.) * np.log(zz) + newlnprob - lnprob0

/home/extmilan/anaconda3/lib/python3.6/site-packages/emcee/ensemble.py:336: RuntimeWarning: invalid value encountered in greater
  accept = (lnpdiff > np.log(self._random.rand(len(lnpdiff))))

snapnr  : 122
11:53:45.242052
[ 32.50867844  31.31370544  35.09283066]
snapnr  : 123
12:08:24.170504
[ 32.82047653  31.61475182  35.45058823]


In [ ]:
#res_arr = res_arr.reshape(4,16)

In [ ]:
### get data and masks out of simulation

# set maximum (3d) Radius of galaxy
maxr_Mpc = 2. * s.galrad #Mpc
#maxr_Mpc = 0.015 #Mpc

## selection masks
# select ALL particles belonging to the main galaxy within maxr
iall, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) )
# select GAS particles belonging to the main galaxy within maxr
igas, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) & (s.type == 0) )
# select DM particles belonging to the main galaxy within maxr
idm, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) & ((s.type == 1) + (s.type == 2) + (s.type == 3) ))
# select STAR particles belonging to the main galaxy within maxr
istars, = np.where( (s.halo == 0) & (s.r() < maxr_Mpc) & (s.r() > 0.) & (s.type == 4) )

# choose which mask to use
mask = iall    

## check if I actually use them........
# get cartesian and cylindrical vectors
# check unit of v_phi (even though not used)
(x_kpc, y_kpc, z_kpc), (vx_kms, vy_kms, vz_kms), rxyz_kpc, rxy_kpc = get_cartesian_vectors(s, sf, mask)
(R_kpc, phi, z_kpc), (vR_kms, vphi, vz_kms) = get_cylindrical_vectors(s, sf, mask)
(x, y, z), (vx, vy, vz), rxyz, rxy = get_cartesian_vectors(s, sf, mask)
(R, phi, z), (vR, vphi, vz) = get_cylindrical_vectors(s, sf, mask)

# select data for each mask
r_dm_kpc = 1000. * s.r()[idm]
r_gas_kpc = 1000. * s.r()[igas]
r_stars_kpc = 1000. * s.r()[istars]
r_all_kpc = 1000. * s.r()[iall]
(R_dm_kpc, phi, z_dm_kpc), (vR_kms, vphi, vz_kms) = get_cylindrical_vectors(s, sf, idm) 
pot_dm_kms2 = s.pot[idm]
pot_gas_kms2 = s.pot[igas]
pot_stars_kms2 = s.pot[istars]
pot_all_kms2 = s.pot[iall]


In [ ]:
plt.plot(r_stars_kpc, pot_stars_kms2, 'b.', alpha = 0.01, label = 'stars')
plt.plot(r_gas_kpc, pot_gas_kms2, 'g.', alpha = 0.01, label = 'gas')
plt.plot(r_dm_kpc, pot_dm_kms2, 'r.', alpha = 0.1, label = 'DM') 
plt.legend()
plt.xlabel('R [kpc]')
plt.ylabel('pot [$(\\frac{\mathrm{km}}{\mathrm{s}})^2]$')

In [ ]:
# dm particle (random) selection
a = np.array(random.sample(list(enumerate(pot_dm_kms2)),150001))
rand_ind = a[:,0].astype(int)
rand_pot_dm_kms2 = a[:, 1]
rand_r_dm_kpc = r_dm_kpc[rand_ind]
rand_R_dm_kpc = R_dm_kpc[rand_ind]
rand_z_dm_kpc = z_dm_kpc[rand_ind]

r_kpc_data, R_kpc_data, z_kpc_data, pot_kms2_data = rand_r_dm_kpc, rand_R_dm_kpc, rand_z_dm_kpc, rand_pot_dm_kms2

plt.plot(r_kpc_data, pot_kms2_data, 'r.', alpha = 0.1, label = 'DM') 
plt.legend()
plt.xlabel('r [kpc]')
plt.ylabel('pot [$(\\frac{\mathrm{km}}{\mathrm{s}})^2]$')
plt.show()
plt.plot(R_kpc_data, pot_kms2_data, 'r.', alpha = 0.1, label = 'DM') 
plt.legend()
plt.xlabel('R [kpc]')
plt.ylabel('pot [$(\\frac{\mathrm{km}}{\mathrm{s}})^2]$')
plt.show()
plt.plot(z_kpc_data, pot_kms2_data, 'r.', alpha = 0.1, label = 'DM') 
plt.legend()
plt.xlabel('z [kpc]')
plt.ylabel('pot [$(\\frac{\mathrm{km}}{\mathrm{s}})^2]$')
plt.show()


In [ ]:
#_____reference values for galpy_____
_REFR0_kpc = 8.

Scipy differential_evolution
----


In [ ]:
#_____function for scipy.differential_evolution that sums the relative error squares_____
def rel_pot_error_scipydifferentialevolution(x,*args):
    '''
    INPUT: x: parameters to be fitted
               x[0]: v0 [km/s]
               x[1], x[2]: scale lengths of Miyamoto Nagai disk [kpc]
               x[3], x[4]: scale length of MFW and Hernquist halo and bulge potentials [kpc]
               x[5], x[6]: normalization of MND and NFWH (including normalization of HB by subtracting them from 1)
           args: data
               args[0], args[1]: grid of R and z in kpc
               args[2]: potential in (km/s)**2
    OUTPUT: err: sum of relative error squares
    '''
    
    #read fitting parameters:
    v0_kms     = x[0]
    a_MND_kpc  = x[1]
    b_MND_kpc  = x[2]
    a_NFWH_kpc = x[3]
    a_HB_kpc   = x[4]
    n_MND      = x[5]
    n_NFWH     = x[6]
    n_HB       = 1.-n_NFWH-n_MND
    
    #read data:
    R_kpc_data    = args[0]
    z_kpc_data    = args[1]
    pot_kms2_data = args[2]
    
    #setup potential (and check if parameters are physical):
    try:
        pot_galpy_model = setup_galpy_potential(a_MND_kpc,b_MND_kpc,a_NFWH_kpc,a_HB_kpc,n_MND,n_NFWH,n_HB)
        #print('Setup potential works.')
    except Exception as e:
        #uncomment for debugging:
        #print(e.message)
        #print(x)
        return np.inf

    #calculate potential values at (R,z) for this potential:
    pot_kms2_model = evaluatePotentials(pot_galpy_model,
                                   R_kpc_data/_REFR0_kpc,
                                   z_kpc_data/_REFR0_kpc) * (v0_kms)**2
    
    #calculate sum of relative error squares:
    err = np.sum(((pot_kms2_data - pot_kms2_model) / pot_kms2_model)**2)
    #print(err)
    return err

In [ ]:
lower_floor = 1e-3
bounds = ((lower_floor,400.), #vcirc
          (lower_floor,200.), #a_MND
          (lower_floor,200.), #b_MND
          (lower_floor,200.), #a_NFWH
          (lower_floor,200.), #a_HB
          (0.,1.), #n_MND
          (0.,1.)) #n_NFWH

#fitting:
res = opt.differential_evolution(
            rel_pot_error_scipydifferentialevolution,
            bounds=bounds,
            args=(R_kpc_data,z_kpc_data,pot_kms2_data),
            strategy='best1bin',
            maxiter=10000,tol=0.001,atol=1e-7,
            disp=False
            )

print(res)
print("best fit:")
print("v0_kms    ", res.x[0])
print("a_MND_kpc ", res.x[1])
print("b_MND_kpc ", res.x[2])
print("a_NFWH_kpc", res.x[3])
print("a_HB_kpc  ", res.x[4])
print("n_MND     ", res.x[5])
print("n_NFWH    ", res.x[6])
print("n_HB      ", 1.-res.x[5]-res.x[6])

pot_diff_ev = np.array([res.x[0], res.x[1], res.x[2], res.x[3], res.x[4], res.x[5], res.x[6],  1.-res.x[5]-res.x[6]])
np.savetxt(outdir + 'pot_res_diff_ev.txt', pot_diff_ev, fmt = '%10.15f')

In [ ]:
v0_kms_bestfit     = res.x[0]
a_MND_kpc_bestfit  = res.x[1]
b_MND_kpc_bestfit  = res.x[2]
a_NFWH_kpc_bestfit = res.x[3]
a_HB_kpc_bestfit   = res.x[4]
n_MND_bestfit      = res.x[5]
n_NFWH_bestfit     = res.x[6]
n_HB_bestfit       = 1. - (n_MND_bestfit + n_NFWH_bestfit)

#generate potential measurements:
pot_galpy_bestfit = setup_galpy_potential(a_MND_kpc_bestfit,b_MND_kpc_bestfit,
                                       a_NFWH_kpc_bestfit,a_HB_kpc_bestfit,
                                       n_MND_bestfit,n_NFWH_bestfit,n_HB_bestfit)
pot_kms2_bestfit = evaluatePotentials(pot_galpy_bestfit,
                                   R_kpc_data/_REFR0_kpc,
                                   z_kpc_data/_REFR0_kpc) * (v0_kms_bestfit)**2

In [ ]:
plotDensities(pot_galpy_bestfit,rmin=00.1,rmax = 6,zmax=6,zmin=-6,nrs=101,nzs=101, log=1, aspect = 'equal')
plt.show()
print('log scale')

Errors of this fit?


emcee fitting
----

In [ ]:
print('Start emcee 1:', datetime.datetime.now().time())

In [ ]:
#_____prepare MCMC_____
ndim, nwalkers, nstep = 7, 50, 10000

#initial values for fit:
v0_kms_init     = 150.
a_MND_kpc_init  = 6.
b_MND_kpc_init  = 1.
a_NFWH_kpc_init = 10.
a_HB_kpc_init   = 2.
n_MND_init      = 0.2
n_NFWH_init     = 0.6
initial_guess = np.array([v0_kms_init,a_MND_kpc_init,b_MND_kpc_init,a_NFWH_kpc_init,a_HB_kpc_init,
                             n_MND_init,n_NFWH_init])
initial_guess_width = np.fabs(0.2*initial_guess)
p0 = np.array([np.random.randn(ndim)*np.array(initial_guess_width)+initial_guess \
                                              for i in range(nwalkers)])
#fiitting boundaries:
boundaries = np.array([[0.,np.inf],[0.,np.inf],[0.,np.inf],[0.,np.inf],[0.,np.inf],[0.,1.],[0.,1.]])
for pp in range(ndim):
    p0[(p0[:,pp] < boundaries[pp,0]),pp] = initial_guess[pp] + np.random.uniform() * 0.1
    p0[(p0[:,pp] > boundaries[pp,1]),pp] = initial_guess[pp] - np.random.uniform() * 0.1

#_____run MCMC_____
sampler = emcee.EnsembleSampler(nwalkers,ndim,
                               lnprob_MCMC,
                               args=[R_kpc_data,z_kpc_data,pot_kms2_data])
sampler.run_mcmc(p0,nstep)

#result:
burnin = 1000
final_samples = sampler.chain[:, burnin:, :].reshape((-1,ndim))
chain_samples = sampler.chain

In [ ]:
#===== OUTPUT =====

#_____take precentiles_____
#perc = np.percentile(final_samples,[16.,50.,84.],axis=0)
median = np.median(final_samples,axis=0)
std = np.std(final_samples,axis=0)
print("best fit:")
print("v0_kms    ",median[0],"\t","+/-",std[0])
print("a_MND_kpc ",median[1],"\t","+/-",std[1])
print("b_MND_kpc ",median[2],"\t","+/-",std[2])
print("a_NFWH_kpc",median[3],"\t","+/-",std[3])
print("a_HB_kpc  ",median[4],"\t","+/-",std[4])
print("n_MND     ",median[5],"\t","+/-",std[5])
print("n_NFWH    ",median[6],"\t","+/-",std[6])
print("n_HB      ",1.-median[5]-median[6])

pot_emcee1 = np.array([median[0], median[1], median[2], median[3], median[4], median[5], median[6], 1. - median[5] - median[6]])
np.savetxt(outdir + 'pot_res_emcee1.txt', pot_emcee1, fmt = '%10.15f')

In [ ]:
#plot progress
labels = ["v0_kms","a_MND_kpc","b_MND_kpc","a_NFWH_kpc","a_HB_kpc","n_MND","n_NFWH","n_HB"]
fig = plt.figure(figsize=(6,12))
ax_list = [fig.add_subplot(ndim,1,ii+1) for ii in range(ndim)]
for ii in range(ndim):
    ax = ax_list[ii]
    ax.set_ylabel(labels[ii])
    for jj in range(nwalkers):
        ax.plot(np.arange(nstep),chain_samples[jj,:,ii],color='k',alpha=0.5,linewidth=0.2)    
for ii in range(ndim):
    ax = ax_list[ii]
    perc = np.percentile(final_samples[:,ii],[50.])
    #ax.hlines(true_values[ii],0.,nstep,color='green',linestyles=['solid'],label='true value')
    ax.hlines(perc,0.,nstep,color='red',linestyles=['dotted'],label='best fit from MCMC')
    ylim = ax.get_ylim()
    ax.vlines([burnin],ylim[0],ylim[1],color='grey',linestyle='dashed',label='burn in')
    ax.set_ylim(ylim)
    if ii == 0: ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
#_____generate data from best fit potential_____
#bestfit:
v0_kms_bestfit     = median[0]
a_MND_kpc_bestfit  = median[1]
b_MND_kpc_bestfit  = median[2]
a_NFWH_kpc_bestfit = median[3]
a_HB_kpc_bestfit   = median[4]
n_MND_bestfit      = median[5]
n_NFWH_bestfit     = median[6]
n_HB_bestfit       = 1. - (n_MND_bestfit + n_NFWH_bestfit)

#generate potential measurements:
pot_galpy_bestfit = setup_galpy_potential(a_MND_kpc_bestfit,b_MND_kpc_bestfit,
                                       a_NFWH_kpc_bestfit,a_HB_kpc_bestfit,
                                       n_MND_bestfit,n_NFWH_bestfit,n_HB_bestfit)
pot_kms2_bestfit = evaluatePotentials(pot_galpy_bestfit,
                                   R_kpc_data/_REFR0_kpc,
                                   z_kpc_data/_REFR0_kpc) * (v0_kms_bestfit)**2

In [ ]:
#_____plot density distribution of best fit_____
plotDensities(pot_galpy_bestfit,rmin=0.01,rmax = 6,zmax=6,zmin=-6,nrs=101,nzs=101, log=1, aspect = 'equal')
plt.show()
print('log scale')

In [ ]:
print('Start emcee 2:', datetime.datetime.now().time())

In [ ]:
#_____prepare MCMC_____
ndim, nwalkers, nstep = 7, 50, 3000

#initial values for fit:
initial_guess = median
initial_guess_width = 0.1*std
p0 = np.array([np.random.randn(ndim)*np.array(initial_guess_width)+initial_guess \
                                              for i in range(nwalkers)])

#_____run MCMC_____
sampler = emcee.EnsembleSampler(nwalkers,ndim,
                               lnprob_MCMC,
                               args=[R_kpc_data,z_kpc_data,pot_kms2_data])
sampler.run_mcmc(p0,nstep)

#result:
burnin = 700
final_samples = sampler.chain[:, burnin:, :].reshape((-1,ndim))
chain_samples = sampler.chain

In [ ]:
#===== OUTPUT =====

#_____take precentiles_____
#perc = np.percentile(final_samples,[16.,50.,84.],axis=0)
median = np.median(final_samples,axis=0)
std = np.std(final_samples,axis=0)
print("true vs. best fit:")
print("v0_kms    ",median[0],"\t","+/-",std[0])
print("a_MND_kpc ",median[1],"\t","+/-",std[1])
print("b_MND_kpc ",median[2],"\t","+/-",std[2])
print("a_NFWH_kpc",median[3],"\t","+/-",std[3])
print("a_HB_kpc  ",median[4],"\t","+/-",std[4])
print("n_MND     ",median[5],"\t","+/-",std[5])
print("n_NFWH    ",median[6],"\t","+/-",std[6])
print("n_HB      ",1.-median[5]-median[6])

pot_emcee2 = np.array([median[0], median[1], median[2], median[3], median[4], median[5], median[6], 1. - median[5] - median[6]])
np.savetxt(outdir + 'pot_res_emcee2.txt', pot_emcee2, fmt = '%10.15f')

In [ ]:
#plot progress
fig = plt.figure(figsize=(6,12))
ax_list = [fig.add_subplot(ndim,1,ii+1) for ii in range(ndim)]
for ii in range(ndim):
    ax = ax_list[ii]
    ax.set_ylabel(labels[ii])
    for jj in range(nwalkers):
        ax.plot(np.arange(nstep),chain_samples[jj,:,ii],color='k',alpha=0.5,linewidth=0.2)    
for ii in range(ndim):
    ax = ax_list[ii]
    perc = np.percentile(final_samples[:,ii],[50.])
    #ax.hlines(true_values[ii],0.,nstep,color='green',linestyles=['solid'],label='true value')
    ax.hlines(perc,0.,nstep,color='red',linestyles=['dotted'],label='best fit from MCMC')
    ylim = ax.get_ylim()
    ax.vlines([burnin],ylim[0],ylim[1],color='grey',linestyle='dashed',label='burn in')
    ax.set_ylim(ylim)
    if ii == 0: ax.legend()
plt.tight_layout()
plt.show()

In [ ]:
#_____generate data from best fit potential_____
#bestfit:
v0_kms_bestfit     = median[0]
a_MND_kpc_bestfit  = median[1]
b_MND_kpc_bestfit  = median[2]
a_NFWH_kpc_bestfit = median[3]
a_HB_kpc_bestfit   = median[4]
n_MND_bestfit      = median[5]
n_NFWH_bestfit     = median[6]
n_HB_bestfit       = 1. - (n_MND_bestfit + n_NFWH_bestfit)

#generate potential measurements:
pot_galpy_bestfit = setup_galpy_potential(a_MND_kpc_bestfit,b_MND_kpc_bestfit,
                                       a_NFWH_kpc_bestfit,a_HB_kpc_bestfit,
                                       n_MND_bestfit,n_NFWH_bestfit,n_HB_bestfit)
pot_kms2_bestfit = evaluatePotentials(pot_galpy_bestfit,
                                   R_kpc_data/_REFR0_kpc,
                                   z_kpc_data/_REFR0_kpc) * (v0_kms_bestfit)**2

In [ ]:
#_____plot density distribution of best fit_____
plotDensities(pot_galpy_bestfit,rmin=0.01,rmax = 6,zmax=6,zmin=-6,nrs=101,nzs=101, log=1, aspect = 'equal')
plt.show()
print('log scale')

In [ ]:
print('End emcee 2:', datetime.datetime.now().time())